In [33]:
import pandas as pd
import matplotlib as plt
import requests
from datetime import timedelta

GitHubUserURLBase = "https://api.github.com/users/"
GitHubPublicEvents = "/events/public"

class GitHubUser:
    """
    Creates and github user object
    method get_github_user_activity gets all public activity for a user for the last 90 days max 300 events
    """
    def __init__(self, username):
        self.username = username
    def get_github_user(self):
        """
        method get_github_user gets user details
        """
        UserInfo = requests.get(GitHubUserURLBase + self.username)
        UserInfoUsable = UserInfo.json()
        return(UserInfoUsable)
    def verify_github_user(self):
        """
        method verify_github_user verifies the user exists
        returns a error if user doesn't exist
        """
        UserInfo = self.get_github_user()
        if 'login' in UserInfo:
            return("User_Exists")
        else:
            raise ValueError("Not a valid GitHub user")
    def get_github_user_activity(self):
        """
        returns the public activity for a user
        """
        UserActivityList = []
        PageNumber = 1
        while PageNumber <= 10:
            PageNumberStr = str(PageNumber)
            print("Getting events Page " + PageNumberStr + " of 10")
            UserActivity = requests.get(GitHubUserURLBase + self.username + GitHubPublicEvents + "?page=" + PageNumberStr)
            UserActivityPage = UserActivity.json()
            print(UserActivityPage)
            UserActivityList = UserActivityList + UserActivityPage
            PageNumber += 1
        print(len(UserActivityList))
        return(UserActivityList)

User = GitHubUser("detaer")
User.get_github_user_activity()
    
def UserActivityFrame(userevents):
    ids = []
    types = []
    times = []
    for activity in userevents:
        ids.append(activity['id'])
        types.append(activity['type'])
        times.append(activity['created_at'])
    df = pd.DataFrame({'id' : ids,
         'type' : types,
         'created_at' : times})
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['created_at'] + timedelta(hours = 8)
    df['hour_of_day'] = df.created_at.dt.hour
    df['day_of_week'] = df.created_at.dt.dayofweek
    return(df)


# def main():
#     UserName = str(input("Enter the GitHub username of the person you want to creep on:"))
#     User = GitHubUser(UserName)
#     ##do a try except to throw an error if the user isn't real, if the user is real, grab their public activity
#     try:
#         TestUser = User.verify_github_user()
#         print(TestUser)
#         print("Working!")
#         UserEvents = User.get_github_user_activity()
#         UserEventsDF = UserActivityFrame(UserEvents)
#         print(UserEventsDF)
#     except:
#         print("User is not a valid github user!")    

# if __name__ == '__main__':
#     main()


Getting events Page 1 of 10
{'message': "API rate limit exceeded for 136.24.165.200. (But here's the good news: Authenticated requests get a higher rate limit. Check out the documentation for more details.)", 'documentation_url': 'https://developer.github.com/v3/#rate-limiting'}


TypeError: can only concatenate list (not "dict") to list

In [ ]:
%matplotlib inline